In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import time

from utils.file_dialog_utils import pick_media_cv2
from utils.file_dialog_utils import pick_folder

# 1. Load target (image or video)

# Image or Video (target source)
media_obj, media_path, source_kind = pick_media_cv2(title="Select a wheel frame image or video")

if source_kind == "image":
    wheel_bgr = media_obj
    wheel_path = media_path
    if wheel_bgr is None:
        raise ValueError("Check your file paths - the image did not load.")
    # Convert to grayscale (matchTemplate works well on grayscale)
    wheel_gray = cv2.cvtColor(wheel_bgr, cv2.COLOR_BGR2GRAY)
else:
    wheel_cap = media_obj  # cv2.VideoCapture
    wheel_path = media_path
    if wheel_cap is None or not wheel_cap.isOpened():
        raise ValueError("Check your file paths - the video did not open.")


In [2]:
# 2. Prepare rotated templates and (if image) find best overall

# Folder where your rotated templates are saved
templates_dir = pick_folder(title="Select folder with rotated templates")

template_paths = sorted(glob.glob(os.path.join(templates_dir, "*.png")))
print(f"Found {len(template_paths)} templates")

if not template_paths:
    raise ValueError("No rotated templates found. Check templates_dir.")

print("Starting template matching...")
time.sleep(2)

method = cv2.TM_CCOEFF_NORMED

if source_kind == "image":
    # Find best template across all rotations for the single image
    best = {
        "score": -1.0,
        "path": None,
        "top_left": None,
        "w": None,
        "h": None,
        "result": None,
        "template_bgr": None,
    }

    for tpath in template_paths:
        tmpl_bgr = cv2.imread(tpath)
        tmpl_gray = cv2.cvtColor(tmpl_bgr, cv2.COLOR_BGR2GRAY)

        h, w = tmpl_gray.shape[:2]

        res = cv2.matchTemplate(wheel_gray, tmpl_gray, method)
        _, max_val, _, max_loc = cv2.minMaxLoc(res)

        print(f"[{template_paths.index(tpath)+1}/{len(template_paths)}] {os.path.basename(tpath)} | score={max_val:.4f} || best={best['score']:.4f}")

        # Track best overall
        if max_val > best["score"]:
            best.update({
                "score": max_val,
                "path": tpath,
                "top_left": max_loc,
                "w": w,
                "h": h,
                "result": res,
                "template_bgr": tmpl_bgr.copy(),
            })

        # Early stopping if perfect match found
        if max_val >= 0.99:
            print("Early stopping: perfect match found.")
            break

    print("\n=== Final best match ===")
    print(f"Template: {best['path']}")
    print(f"Score:    {best['score']:.4f}")
else:
    # Preload templates for faster per-frame matching on video
    templates_preloaded = []
    for tpath in template_paths:
        tmpl_bgr = cv2.imread(tpath)
        if tmpl_bgr is None:
            print(f"Warning: failed to load template: {tpath}")
            continue
        tmpl_gray = cv2.cvtColor(tmpl_bgr, cv2.COLOR_BGR2GRAY)
        h, w = tmpl_gray.shape[:2]
        templates_preloaded.append({
            "path": tpath,
            "gray": tmpl_gray,
            "w": w,
            "h": h,
        })

    print(f"Preloaded {len(templates_preloaded)} templates for video processing")


Found 80 templates
Starting template matching...
Preloaded 80 templates for video processing


In [3]:
# 3. Visualize for image OR render annotated video for video

if source_kind == "image":
    top_left = best["top_left"]
    w, h = best["w"], best["h"]
    bottom_right = (top_left[0] + w, top_left[1] + h)

    wheel_vis = wheel_bgr.copy()
    cv2.rectangle(wheel_vis, top_left, bottom_right, (0, 255, 0), 2)

    wheel_vis_rgb = cv2.cvtColor(wheel_vis, cv2.COLOR_BGR2RGB)
    template_rgb = cv2.cvtColor(best["template_bgr"], cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(12, 4))

    plt.subplot(1, 3, 1)
    plt.imshow(template_rgb)
    plt.title(f"Best Template\n{os.path.basename(best['path'])}")
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.imshow(wheel_vis_rgb)
    plt.title(f"Best Match (score={best['score']:.3f})")
    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.imshow(best["result"], cmap="hot")
    plt.title("Match Heatmap")
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.axis("off")

    plt.tight_layout()
    plt.show()
else:
    total_frames = int(wheel_cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
    print("Total frames in video:", total_frames)
    print("Templates:", len(templates_preloaded))
    print(f"Template Comparisons = {total_frames} x {len(templates_preloaded)} = {total_frames * len(templates_preloaded)}")

    # Render an annotated video showing the best match per frame
    fps = wheel_cap.get(cv2.CAP_PROP_FPS) or 25.0
    width = int(wheel_cap.get(cv2.CAP_PROP_FRAME_WIDTH) or 0)
    height = int(wheel_cap.get(cv2.CAP_PROP_FRAME_HEIGHT) or 0)

    if width <= 0 or height <= 0:
        # Fallback: read one frame to infer size
        ok, frame = wheel_cap.read()
        if not ok:
            raise RuntimeError("Failed to read from video to infer size")
        height, width = frame.shape[:2]
        wheel_cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

    # Choose an output path next to the input
    base_dir = os.path.dirname(wheel_path)
    out_path_mp4 = os.path.join(base_dir, "matched_output.mp4")

    # Try mp4 first, then fallback to avi if needed
    fourcc_mp4 = cv2.VideoWriter_fourcc(*"mp4v")
    writer = cv2.VideoWriter(out_path_mp4, fourcc_mp4, fps, (width, height))

    if not writer.isOpened():
        out_path_avi = os.path.join(base_dir, "matched_output.avi")
        fourcc_avi = cv2.VideoWriter_fourcc(*"XVID")
        writer = cv2.VideoWriter(out_path_avi, fourcc_avi, fps, (width, height))
        out_path = out_path_avi
    else:
        out_path = out_path_mp4

    frame_idx = 0
    while True:
        ok, frame_bgr = wheel_cap.read()
        if not ok:
            break

        frame_gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)

        # Find best match across all rotated templates for this frame
        best_score = -1.0
        best_loc = None
        best_size = None

        for t in templates_preloaded:
            res = cv2.matchTemplate(frame_gray, t["gray"], method)
            _, max_val, _, max_loc = cv2.minMaxLoc(res)
            if max_val > best_score:
                best_score = max_val
                best_loc = max_loc
                best_size = (t["w"], t["h"])  # (w, h)

        if best_loc is not None and best_size is not None:
            w, h = best_size
            top_left = best_loc
            bottom_right = (top_left[0] + w, top_left[1] + h)
            cv2.rectangle(frame_bgr, top_left, bottom_right, (0, 255, 0), 2)
            cv2.putText(
                frame_bgr,
                f"score={best_score:.3f}",
                (top_left[0], max(0, top_left[1]-5)),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 255, 0),
                1,
                cv2.LINE_AA,
            )

        writer.write(frame_bgr)
        frame_idx += 1

    writer.release()
    wheel_cap.release()

    print(f"Annotated video saved to: {out_path}")
    
    # Normalize path for HTML display
    disp_path = out_path.replace("\\", "/")


Total frames in video: 333
Templates: 80
Template Comparisons = 333 x 80 = 26640
Annotated video saved to: C:/Users/Gabriel/Documents/Dissertation/Code/data\matched_output.mp4


In [4]:
# 4. Multiple matches (image only)

if source_kind == "image":
    threshold = 0.6
    result = best["result"]
    w, h = best["w"], best["h"]

    if method in [cv2.TM_CCOEFF_NORMED, cv2.TM_CCORR_NORMED]:
        loc = np.where(result >= threshold)
        wheel_multi = wheel_bgr.copy()

        for pt_y, pt_x in zip(*loc):
            top_left_multi = (pt_x, pt_y)
            bottom_right_multi = (pt_x + w, pt_y + h)
            cv2.rectangle(wheel_multi, top_left_multi, bottom_right_multi, (255, 0, 0), 1)

        wheel_multi_rgb = cv2.cvtColor(wheel_multi, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(6, 6))
        plt.imshow(wheel_multi_rgb)
        plt.title(f"Matches with score >= {threshold}")
        plt.axis("off")
        plt.show()
else:
    print("Multiple match visualization is not implemented for video mode.")


Multiple match visualization is not implemented for video mode.
